from __future__ import print_function
from __future__ import division
import cv2 as cv
import numpy as np
# Create an image
r = 100
src = np.zeros((4*r, 4*r), dtype=np.uint8)
# Create a sequence of points to make a contour
vert = [None]*6
vert[0] = (3*r//2, int(1.34*r))
vert[1] = (1*r, 2*r)
vert[2] = (3*r//2, int(2.866*r))
vert[3] = (5*r//2, int(2.866*r))
vert[4] = (3*r, 2*r)
vert[5] = (5*r//2, int(1.34*r))
# Draw it in src
for i in range(6):
    cv.line(src, vert[i],  vert[(i+1)%6], ( 255 ), 3)
# Get the contours
contours, _ = cv.findContours(src, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
print(contours)
# Calculate the distances to the contour
raw_dist = np.empty(src.shape, dtype=np.float32)
for i in range(src.shape[0]):
    for j in range(src.shape[1]):
        raw_dist[i,j] = cv.pointPolygonTest(contours[0], (j,i), True)

minVal, maxVal, _, maxDistPt = cv.minMaxLoc(raw_dist)
minVal = abs(minVal)
maxVal = abs(maxVal)
# Depicting the  distances graphically
drawing = np.zeros((src.shape[0], src.shape[1], 3), dtype=np.uint8)
for i in range(src.shape[0]):
    for j in range(src.shape[1]):
        if raw_dist[i,j] < 0:
            drawing[i,j,0] = 255 - abs(raw_dist[i,j]) * 255 / minVal
        elif raw_dist[i,j] > 0:
            drawing[i,j,2] = 255 - raw_dist[i,j] * 255 / maxVal
        else:
            drawing[i,j,0] = 255
            drawing[i,j,1] = 255
            drawing[i,j,2] = 255
cv.circle(drawing,maxDistPt, int(maxVal),(255,255,255), 1, cv.LINE_8, 0)
cv.imshow('Source', src)
cv.imshow('Distance and inscribed circle', drawing)
cv.waitKey()

In [1]:
# Copyright 2018 Changan Wang

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# =============================================================================
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import math
import tensorflow as tf
# from scipy.misc import imread, imsave, imshow, imresize
import numpy as np
from tensorflow.python.tools import freeze_graph
#from net import ssd_net#_48 as ssd_net

from dataset import dataset_common
from preprocessing import ssd_preprocessing
from utility import anchor_manipulator
from utility.selectROINEW import selectROINEW
from utility import draw_toolbox
from numpy import exp, abs, angle
import time
import cv2 as cv
from make_npy import make_npy
from easydict import EasyDict as edict
import json
import copy 

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
class CusTrackers:
   # def __init__(self):
        

   # def __del__(self):
        

    class perTracker:
        
        def __init__(self,no,box,frame,tracker_type,typef):
            self.lostTrackCnt=0
            self.noDetectionCnt=0
            self.detectedcnt=0
            self.no=no
            self.box=box
            self.historylength=10
            self.patternmatchingcnt=0
            self.pattern=[-1,-1,-1] #average of b,g,r value
            self.boxmidpointx=(box[0]+box[0]+box[2])/2
            self.boxmidpointy=(box[1]+box[1]+box[3])/2
            boxmidtoFramemidRange=math.sqrt((frame.shape[0]/2-self.boxmidpointy)*(frame.shape[0]/2-self.boxmidpointy)
                                   +(frame.shape[1]/2-self.boxmidpointx)*(frame.shape[1]/2-self.boxmidpointx))
            ratio=(box[2]/2)/boxmidtoFramemidRange
            pointneartomidx=int(self.boxmidpointx+(frame.shape[1]/2-self.boxmidpointx)*ratio)
            pointneartomidy=int(self.boxmidpointy+(frame.shape[0]/2 - self.boxmidpointy )*ratio)

            self.locationhistory=[[pointneartomidx,pointneartomidy]]
     
            
            self.crop=None
            print(tracker_type)
            if tracker_type == 'BOOSTING':
                self.cvtracker = cv.TrackerBoosting_create()
            if tracker_type == 'MIL':
                self.cvtracker = cv.TrackerMIL_create()
            if tracker_type == 'KCF':
                self.cvtracker = cv.TrackerKCF_create()
            if tracker_type == 'TLD':
                self.cvtracker = cv.TrackerTLD_create()
            if tracker_type == 'MEDIANFLOW':
                self.cvtracker = cv.TrackerMedianFlow_create()
            if tracker_type == 'GOTURN':
                self.cvtracker = cv.TrackerGOTURN_create()
            if tracker_type == 'MOSSE':
                self.cvtracker = cv.TrackerMOSSE_create()
            if tracker_type == "CSRT":
                self.cvtracker = cv.TrackerCSRT_create()
            #self.cvtracker.save("default_"+str(typef)+".xml")
            fs = cv.FileStorage("default_"+str(typef)+".xml", cv.FileStorage_READ)
            self.cvtracker.read(fs.getFirstTopLevelNode())
            box=tuple(box)
            print(box)
            self.cvtracker.init(frame, box)     
        #def __del__(self):
            #del self.cvtracker
        def update(self,frame):
            if(self.lostTrackCnt==0):
                ok, tbox = self.cvtracker.update(frame)
               
                if ok:
                    self.box=tbox
                    if len(self.locationhistory)>=self.historylength:
                        for i in range(len(self.locationhistory)-self.historylength+1):
                            del self.locationhistory[0]
                    box=tbox
                    boxmidpointx=(box[0]+box[0]+box[2])/2
                    boxmidpointy=(box[1]+box[1]+box[3])/2
                    boxmidtoFramemidRange=math.sqrt((frame.shape[0]/2-boxmidpointy)*(frame.shape[0]/2-boxmidpointy)
                                           +(frame.shape[1]/2-boxmidpointx)*(frame.shape[1]/2-boxmidpointx))
                    ratio=(box[2]/2)/boxmidtoFramemidRange
                    pointneartomidx=int(boxmidpointx+(frame.shape[1]/2-boxmidpointx)*ratio)
                    pointneartomidy=int(boxmidpointy+(frame.shape[0]/2 - boxmidpointy )*ratio)
                    
                    self.locationhistory.append([pointneartomidx,pointneartomidy])
                else:
                    self.lostTrackCnt=1
                    #self.pattern_refresh(frame)
                    #del self.cvtracker
                    self.box=self.box
                    box=self.box
                    if len(self.locationhistory)>=self.historylength:
                        for i in range(len(self.locationhistory)-self.historylength+1):
                            del self.locationhistory[0]
                    boxmidpointx=(box[0]+box[0]+box[2])/2
                    boxmidpointy=(box[1]+box[1]+box[3])/2
                    boxmidtoFramemidRange=math.sqrt((frame.shape[0]/2-boxmidpointy)*(frame.shape[0]/2-boxmidpointy)
                                           +(frame.shape[1]/2-boxmidpointx)*(frame.shape[1]/2-boxmidpointx))
                    ratio=(box[2]/2)/boxmidtoFramemidRange
                    pointneartomidx=int(boxmidpointx+(frame.shape[1]/2-boxmidpointx)*ratio)
                    pointneartomidy=int(boxmidpointy+(frame.shape[0]/2 - boxmidpointy )*ratio)
                    
                    self.locationhistory.append([pointneartomidx,pointneartomidy])
            else:
                self.lostTrackCnt+=1
            return self.box,self.lostTrackCnt
        def pattern_refresh(self,frame):
            for notint in self.box:
                notint=int(notint)

            self.crop = frame[int(self.box[1]):int(self.box[1]+self.box[3]), int(self.box[0]):int(self.box[0]+self.box[2])]
            
            
        def pattern_matching(self,frame):
            res = cv.matchTemplate(frame,self.crop,cv.TM_CCOEFF_NORMED)
            #print('tryfind match')
            if(np.amax(res)  >0.5):
               # print('pattern match')
                min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
     
                top_left = max_loc
               # print(self.box[0])
               # print(max_loc[0])
                self.box = (max_loc[0],max_loc[1],self.box[2],self.box[3])

    class perDetectContour:

        def __init__(self,ContourPts):
            
            self.trackerno=-1
            self.contour=ContourPts
    class occulusionedBox:

        def __init__(self,no,box,pattern):
            self.lastedcnt=0
            self.no=no
            self.box=box
            self.pattern=pattern
    class perBorder:

        def __init__(self,leftpoint,rightpoint):
            self.leftpoint=leftpoint
            self.inoutcount=0 #in+=1 out-=1
            self.rightpoint=rightpoint
            self.ConsiderRange=CusTrackers.borderConsiderRange
            cc1x,cc1y,c1x,c1y=self.getPerpCoord(self.leftpoint[0],self.leftpoint[1],self.rightpoint[0],self.rightpoint[1],self.ConsiderRange)
            c2x,c2y,cc2x,cc2y=self.getPerpCoord(self.rightpoint[0],self.rightpoint[1],self.leftpoint[0],self.leftpoint[1],self.ConsiderRange)
            #print('cc1x,cc1y,c1x,c1y',cc1x,cc1y,c1x,c1y)
            #print('c2x,c2y,cc2x,cc2y',c2x,c2y,cc2x,cc2y)
           
            area1=[self.leftpoint,self.rightpoint,[cc1x,cc1y],[cc2x,cc2y]]
            area2=[self.rightpoint,self.leftpoint,[c2x,c2y],[c1x,c1y]]
            area1midpoint=[(self.leftpoint[0]+cc1x)/2,(self.leftpoint[1]+cc1y)/2]
            area2midpoint=[(self.leftpoint[0]+c1x)/2,(self.leftpoint[1]+c1y)/2]
            
            if self.getDistance(area1midpoint,[CusTrackers.frame.shape[1]/2,CusTrackers.frame.shape[0]/2])<self.getDistance(area2midpoint,[CusTrackers.frame.shape[1]/2,CusTrackers.frame.shape[0]/2]):
                self.insideArea=area1   #four point counter clockwise
                self.outsideArea=area2
            else :
                self.insideArea=area2
                self.outsideArea=area1
            
            
            
            self.statehistory=[0]*10 #0 nothing happens,1 outside to inside, 2 inside to outside
            
            
            self.insideTrackersNow=[]
            self.outsideTrackersNow=[]
            self.insideTrackersHistory=[] #list of list
            self.outsideTrackersHistory=[]
            

        def getPerpCoord(self,aX, aY, bX, bY, length): #a line perpendicular to line(a to b) and with b in the mid 
            print('aX, aY, bX, bY',aX, aY, bX, bY)
            vX = int(bX)-int(aX)
            vY = int(bY)-int(aY)
            print(type(bY),type(aY))
            print('vX,vY ori',vX,vY)
            #print(str(vX)+" "+str(vY))

            mag = math.sqrt(vX*vX + vY*vY)
            
            vX = vX / mag
            vY = vY / mag
            temp = vX
            vX = 0-vY
            vY = temp
            print('vX,vY after',vX,vY)
            cX = bX + vX * length
            cY = bY + vY * length
            dX = bX - vX * length
            dY = bY - vY * length
            
            return int(cX), int(cY), int(dX), int(dY)     #first point is counter clockwise, second is clockwise 
        def getDistance(self,p1,p2):
            print('p1,p2',p1,p2)
            return math.sqrt(((p1[0]-p2[0])*(p1[0]-p2[0]))+((p1[1]-p2[1])*(p1[1]-p2[1])))
        
        def checkIfInArea(self,point,area):
            area=np.asarray(area)
                
            #print(area)
            return cv.pointPolygonTest(area, (point[0], point[1]), False)
            
    typef=7
    tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
    tracker_type = tracker_types[typef]
    
    tracker_type=tracker_type
    totalcount=0
    
    numbertoAssignList=[0]*20
    occulusionTakenList=[0]*20
    patternmatchinglsit=[]
    trackerlist=[]
    occulusionlist=[]
    detectContourList=[]
    borderList=[]  #a boarder is defined by 2 points, the side frame mid point is in is defined as "inside"
    
    lostTrackerThreshold=3
    qualifyCntThreshold=1000 #if detected more than this frame count, then it's qualified for a occulusioned box
    noDetectionCntThreshold=100
    occulusionedThreshold=5 #if occulusioned more frames than this cnt, delete occulusioned
    borderHistoryLength=10
    borderConsiderRange = 0
           #only start to consider trackers if they are inside this range fo the border
  
    frame=None
    
    def updateframe(self,frame):
        print('updateframe')
        CusTrackers.frame=frame
        CusTrackers.borderConsiderRange = self.frame.shape[0]/10
        return
    def clearDetectionContours(self):
        print('addDetectionContours')
        CusTrackers.detectContourList=[] 
    def addDetectionContours(self,contourPts):
        print('addDetectionContours')
        thiscontour=self.perDetectContour(contourPts)
        CusTrackers.detectContourList.append(thiscontour)
    def updateTrackers(self):
        print('updateTrackers')
        frame=CusTrackers.frame
        if(len(CusTrackers.trackerlist)==0):
            return False
        totalcount=len(CusTrackers.trackerlist)
        for tracker in CusTrackers.trackerlist:
            
            updatedbox,trackerlostcnt=tracker.update(frame)    
        for occulusioned in CusTrackers.occulusionlist:
            occulusioned.lastedcnt+=1
            
    def adjustTrackersbyDetection(self):
        #print('len at adjusttrackerstart',len(CusTrackers.trackerlist))
        nearestcontourlist=[]
        standardtrackerboxlist=[]
        overlaplist=[]
        contournolist=[]
        for i in range(len(CusTrackers.trackerlist)):
            tracker=CusTrackers.trackerlist[i]
            nearestcontour,contourno=self.findNearestContour(tracker.box)
            if nearestcontour:
                
                standardtrackerbox=self.getTrackerBoxfromContour(nearestcontour.contour) 
                
            else:
                standardtrackerbox=False
            nearestcontourlist.append(nearestcontour)
            contournolist.append(contourno)
            standardtrackerboxlist.append(standardtrackerbox)
            if(nearestcontour):
                overlaplist.append(self.checkOverlap(nearestcontour.contour,tracker.box))
                
            else:
                overlaplist.append(0)
        for i in range(len(contournolist)):
            if contournolist[i] is False:
                continue
            for j in range(i+1,len(contournolist)):
                if contournolist[j] is False:
                    continue                
                if contournolist[i]==contournolist[j]:
                    if overlaplist[i]>overlaplist[j]:
                        nearestcontourlist[j]=False
                       # print(' nearestcontourlist[j]=False',i,j)
                    else:
                        nearestcontourlist[i]=False
                       # print(' nearestcontourlist[i]=False',i,j)
        
            
        frame=CusTrackers.frame
        print('len at adjusttracker mid',len(CusTrackers.trackerlist))
        for i in range(len(CusTrackers.trackerlist)):
            tracker=CusTrackers.trackerlist[i]
            notfound=1
            contourno=contournolist[i]
            
            standardtrackerbox=standardtrackerboxlist[i]            
                
            nearestcontour=nearestcontourlist[i]
            if(nearestcontour == False):
                print('nearestcontour == False,i',i)
                tracker.noDetectionCnt+=1
                if(tracker.detectedcnt<0):
                    tracker.noDetectionCnt+=50
                continue

                
            #print('box,contourno',tracker.box,nearestcontour.contour)

            
            elif overlaplist[i]>0.5: #found a detection box this tracker belongs
                
                #print('!!!!!!!!!!!foundDetBoxBelongs')
                tracker.detectedcnt+=1
                tracker.noDetectionCnt=0
              #  print('thereareoverlap,i',i)
               # print(len(contournolist))
                
                
                CusTrackers.detectContourList[contournolist[i]].trackerno=tracker.no
                overlapsize=self.intersectionsize(standardtrackerbox,tracker.box)
                overlapSizetoStandard=overlapsize/(standardtrackerbox[2]*standardtrackerbox[3])
                overlapSizetoTrackerBox=overlapsize/(tracker.box[2]*tracker.box[3])
                #print('overlapSizeRatio',overlapSizeRatio)
                if overlapSizetoTrackerBox<0.8 or overlapSizetoStandard<0.8 or tracker.lostTrackCnt>1: #refresh the trackerbox
                    

                    #if self.checkboxExists(standardtrackerbox,0.9)<0:
                    #trackernew=self.perTracker(tracker.no,standardtrackerbox,frame,CusTrackers.tracker_type,CusTrackers.typef)
                   # print('standardtrackerbox',standardtrackerbox)
                    trackernew=self.replaceTracker(tracker,standardtrackerbox)

                    CusTrackers.trackerlist[i]=trackernew
                   # print('adjustTrackersbyDetection  overlapSizetoTrackerBox<0.8')
                   # print('length',len(CusTrackers.trackerlist))
                   # print('index',i)
                   # print('tracker.no',tracker.no)
                   # print('tracker.box',tracker.box)
                    #nearestcontour.trackerno=tracker.no
            elif self.checkPolarNear(standardtrackerbox,tracker.box):
                #print('checkPolarNear,i',i)
                CusTrackers.detectContourList[contournolist[i]].trackerno=tracker.no
                if self.checkboxExists(standardtrackerbox,1,0.9)<0:
                    #trackernew=self.perTracker(tracker.no,standardtrackerbox,frame,CusTrackers.tracker_type,CusTrackers.typef)
                    trackernew=self.replaceTracker(tracker,standardtrackerbox)

                    CusTrackers.trackerlist[i]=trackernew
                  #  print('adjustTrackersbyDetection self.checkPolarNear')
                   # print('tracker.no',tracker.no)
                  #  print('tracker.box',tracker.box)
                    
            else: #there are detection boxes but no overlap
                
                print('no overlap')
                
                tracker.noDetectionCnt+=1
                
                continue                
                
        return
        
    def addTrackersbyDetection(self): #need to add if detection is occulusioned
        print('addtrackers')
        for detection in CusTrackers.detectContourList:
          #  print('detection.trackerno',detection.trackerno)
            if detection.trackerno==-1:
                standardtrackerbox=self.getTrackerBoxfromContour(detection.contour)
                
                aa=self.checkboxExists(standardtrackerbox,1,0.5)
                bb=self.checkboxExists(standardtrackerbox,2,0.5)
                if aa>0: #overlap with tracker
                   
                    detection.trackerno=CusTrackers.trackerlist[aa].no
                    CusTrackers.trackerlist[aa].noDetectionCnt=0
                elif bb>0: #overlap with occu box
                    #check if pattern match
                    #print('standardtrackerbox',standardtrackerbox)
                    if checkPatternMatch(CusTrackers.occulusionlist[bb].pattern,getpattern(standardtrackerbox)): #the detected box is the
                        detection.trackerno=CusTrackers.occulusionlist[bb].no
                        tracker=self.addTracker(standardtrackerbox)
                        tracker.no==CusTrackers.occulusionlist[bb].no
                        deleteOcculusion(bb)                   
                        
                    
                else: 
                    tracker=self.addTracker(standardtrackerbox)
                    if(tracker):
                      #  print('add1tracker',standardtrackerbox)
                        detection.trackerno=tracker.no
                        
        return
    
    
    
        
    def nms(self, overlapThresh=0.2):
        # if there are no boxes, return an empty list
        if len(CusTrackers.trackerlist) == 0:
            return []
        # if the bounding boxes integers, convert them to floats --
        # this is important since we'll be doing a bunch of divisions
        #print('nms1!!!!!!!!!!!')
        boxes=[]
        detectedcnt=[]
        no=[]
        for tracker in CusTrackers.trackerlist:
            box=[tracker.box[0],tracker.box[1],tracker.box[0]+tracker.box[2],tracker.box[1]+tracker.box[3]]
            detectedcnt.append(tracker.detectedcnt)
            boxes.append(box)
            no.append(tracker.no)
        detectedcnt=np.asarray(detectedcnt)
        
        boxes=np.asarray(boxes)
        #print('nodetection',nodetection)
        #print('no',no)
        if boxes.dtype.kind == "i":
            boxes = boxes.astype("float")
        # initialize the list of picked indexes	
        pick = []
        # grab the coordinates of the bounding boxes
        
        x1 = boxes[:,0]
        y1 = boxes[:,1]
        x2 = boxes[:,2]
        y2 = boxes[:,3]
        # compute the area of the bounding boxes and sort the bounding
        # boxes by the bottom-right y-coordinate of the bounding box
        area = (x2 - x1 + 1) * (y2 - y1 + 1)
        
        idxs = np.argsort(detectedcnt)
        #idxs=np.flip(idxs, 0)
       
     #   print('idxs',idxs)
        # keep looping while some indexes still remain in the indexes
        # list
        while len(idxs) > 0:
            # grab the last index in the indexes list and add the
            # index value to the list of picked indexes
            last = len(idxs) - 1
            i = idxs[last]
            #pick.append(i)
            # find the largest (x, y) coordinates for the start of
            # the bounding box and the smallest (x, y) coordinates
            # for the end of the bounding box
            xx1 = np.maximum(x1[i], x1[idxs[:last]])
            yy1 = np.maximum(y1[i], y1[idxs[:last]])
            xx2 = np.minimum(x2[i], x2[idxs[:last]])
            yy2 = np.minimum(y2[i], y2[idxs[:last]])
            # compute the width and height of the bounding box
            w = np.maximum(0, xx2 - xx1 + 1)
            h = np.maximum(0, yy2 - yy1 + 1)
            # compute the ratio of overlap
            overlap = np.maximum((w * h) / area[idxs[:last]],(w * h) / area[idxs[last]])
         #   print('overlap,i',overlap,i)
            # delete all indexes from the index list that have
            idxbefore=idxs[:]
            idxs = np.delete(idxs, np.concatenate(([last],
                np.where(overlap > overlapThresh)[0])))
            if len(idxbefore) == (len(idxs)+1):
                pick.append(i)
            else:
                listopickone=[i]
                for idxb in idxbefore :
                    if idxb in idxbefore and idxb not in idxs and idxb !=i: # i deleted idxb
                        if CusTrackers.trackerlist[idxb].detectedcnt > CusTrackers.qualifyCntThreshold:   #check if the detected cnt of idxb higer than the threshold
                            listopickone.append(idxb)
                
                fronttrackerno,_=self.occulusionstart(listopickone) #return the tracker at the front 
                pick.append(fronttrackerno)
                            #pick.append(idxb)
                           
                        #pick.append(idxb)
           # print('pick',pick)
           # print('idxs',idxs)
        # return only the bounding boxes that were picked using the
        # integer data type
        deletelist=list(range(len(CusTrackers.trackerlist)))
        for index in sorted(pick, reverse=True):
            del deletelist[index]
     #   print('deletelist',deletelist)
        for index in sorted(deletelist, reverse=True):
          #  print('index',index)
            CusTrackers.numbertoAssignList[CusTrackers.trackerlist[index].no]=0
            for i in range(len(CusTrackers.trackerlist)):
                ggr=0
               # print('no',CusTrackers.trackerlist[i].no)
          #  print('nms:CusTrackers.numbertoAssignList[CusTrackers.trackerlist[index].no] ',CusTrackers.trackerlist[index].no)
          #  print('len(CusTrackers.trackerlist[index])',len(CusTrackers.trackerlist))
            del CusTrackers.trackerlist[index]
          #  print('len(CusTrackers.trackerlist[index])',len(CusTrackers.trackerlist))
          #  print('nms:CusTrackers.numbertoAssignList',CusTrackers.numbertoAssignList)
            
            
        return    
    def recordPatternofNear(self):
        print('recordpattern')
        for i in range(len(CusTrackers.trackerlist)):

            print('no',CusTrackers.trackerlist[i].no)
        for trackers in CusTrackers.trackerlist:
            trackers.pattern=[-1,-1,-1]
        for i in range(len(CusTrackers.trackerlist)):
            for j in range(i,len(CusTrackers.trackerlist)):
                
                trackeri=CusTrackers.trackerlist[i]
                trackerj=CusTrackers.trackerlist[j]
                rangebetween=self.rangebetweenpoints([trackeri.boxmidpointx,trackeri.boxmidpointy]
                                                     ,[trackerj.boxmidpointx,trackerj.boxmidpointy])
                if rangebetween <(trackeri.box[2]+trackerj.box[2])*0.5*1.2:
                

                    if trackeri.pattern==[-1,-1,-1]:
                        box=trackeri.box
                        CusTrackers.trackerlist[i].pattern=self.getpattern(box)
                      
                    if trackerj.pattern==[-1,-1,-1]:
                        box=trackerj.box

                        CusTrackers.trackerlist[j].pattern=self.getpattern(box)
                                                                                                                                                     
                    
    def retireTrackersAndOcculusion(self):
        print('retireTrackers')
        for i in range(len(CusTrackers.trackerlist)):
            ggrr=0
            #print('no',CusTrackers.trackerlist[i].no)
        for i in range(len(CusTrackers.trackerlist)-1,-1,-1):
            tracker=CusTrackers.trackerlist[i]
            if tracker.lostTrackCnt>=CusTrackers.lostTrackerThreshold or tracker.noDetectionCnt>=CusTrackers.noDetectionCntThreshold:
                CusTrackers.numbertoAssignList[tracker.no]=0
               # print('retireTrackersAndOcculusion(self): CusTrackers.numbertoAssignList[tracker.no]=0',tracker.no)
                for detection in CusTrackers.detectContourList:
                    if detection.trackerno==tracker.no:
                        detection.trackerno=-1
                    
                del CusTrackers.trackerlist[i]
        print('retireoccu')
        for i in range(len(CusTrackers.trackerlist)):
            ggege=0
            #print('no',CusTrackers.trackerlist[i].no)
        for i in range(len(CusTrackers.occulusionlist)-1,-1,-1):
            if CusTrackers.occulusionlist[i].lastedcnt>CusTrackers.occulusionedThreshold:
                self.deleteOcculusion(i)
     
    def updateBorders(self):

        for border in CusTrackers.borderList:
            border.insideTrackersNow=[]
            border.outsideTrackersNow=[]
            p1=np.array(border.leftpoint)
            p2=np.array(border.rightpoint)
            if len(border.insideTrackersHistory)== CusTrackers.borderHistoryLength:
                for i in range(len(border.insideTrackersHistory)-1):
                    border.insideTrackersHistory[i]= border.insideTrackersHistory[i+1]
                    border.outsideTrackersHistory[i]= border.outsideTrackersHistory[i+1]
                del border.insideTrackersHistory[-1]
                del border.outsideTrackersHistory[-1]  

            for tracker in CusTrackers.trackerlist:
              #  print('len(trackerlist)',len(CusTrackers.trackerlist))
                p3=np.array([tracker.boxmidpointx,tracker.boxmidpointy])
               # print('p1,p2,p3',p1,p2,p3)
                d=abs(np.cross(p2-p1,p3-p1)/np.linalg.norm(p2-p1))
               # print('no,d',tracker.no,d)
               # print(CusTrackers.borderConsiderRange/10)
                if d>=CusTrackers.borderConsiderRange/30 and d<CusTrackers.borderConsiderRange*2:#not couting the points near the lines
                   # print('no, border.checkIfInArea(p3,border.insideArea)',tracker.no,border.checkIfInArea(p3,border.insideArea))
                   # print('no, border.checkIfInArea(p3,border.outsideArea)',tracker.no,border.checkIfInArea(p3,border.outsideArea))
                    if border.checkIfInArea(p3,border.insideArea)>0:
                        


                        border.insideTrackersNow.append(tracker)
                    elif border.checkIfInArea(p3,border.outsideArea)>0:
                        border.outsideTrackersNow.append(tracker)   
            border.insideTrackersHistory.append(border.insideTrackersNow)
            border.outsideTrackersHistory.append(border.outsideTrackersNow)
            
            #now define "cross the line"
            if len(border.insideTrackersHistory)<10:
                continue
            for thistracker in border.insideTrackersNow:
                outtoin=0
                
                thisno=thistracker.no
                #check if inside "only" for 2 frame, and outside for 3 frames in 5 frames before last 2 frames
                for trackerin in border.insideTrackersHistory[-2]:
                    threeInARow=0
                    if trackerin.no ==thisno: 
                        for trackerin in border.insideTrackersHistory[-3]:
                            
                            if trackerin.no ==thisno: 
                                threeInARow=1
                                break
                        if threeInARow==1:
                            break
                        
                            
                        #check if out side for 3 frame in 5 frames before 2 frames [-3] to [-7]
                        print('2 inside no in a row trackerin.no',trackerin.no)
                        appearInOutCnt=0
                        PartTrackersHistory=border.outsideTrackersHistory[-8:-2] #6 frames
                        for trackersout in PartTrackersHistory:
                            for trackerout in trackersout:
                                if thisno==trackerout.no:
                                    appearInOutCnt+=1
                                if appearInOutCnt>=4:
                                    break
                        print('appearInOutCnt',appearInOutCnt)
                        if appearInOutCnt>=3:
                            outtoin=1
                            border.inoutcount+=1
                            break
                            
            for thistracker in border.outsideTrackersNow:
                intoout=0
                
                thisno=thistracker.no
                #check if inside "only" for 2 frame, and outside for 3 frames in 5 frames before last 2 frames
                for trackerout in border.outsideTrackersHistory[-2]:
                    threeInARow=0
                    print('2 outside no in a row trackerin.no',trackerout.no)
                    if trackerout.no ==thisno: 
                        
                        for trackerin in border.outsideTrackersHistory[-3]:
                            
                            if trackerin.no ==thisno: 
                                threeInARow=1
                                break
                        if threeInARow==1:
                            break
                        
                        
                        #check if out side for 3 frame in 5 frames before 2 frames [-3] to [-7]
                        appearInInCnt=0
                        
                        PartTrackersHistory=border.insideTrackersHistory[-8:-2] #6 frames
                        for trackersin in PartTrackersHistory:
                            for trackerin in trackersin:
                                if thisno==trackerin.no:
                                    appearInInCnt+=1
                                if appearInInCnt>=4:
                                    break
                        print('appearInInCnt',appearInInCnt)
                        if appearInInCnt>=3:
                            intoout=1
                            border.inoutcount-=1
                            break            
                        
                        
                    
            
            
                
    def getpattern(self,box):
        #for now it's just the average value of rgb
        recordi=[0,0,0]
        totali=[0,0,0]
        cnti=0
       # print('box[0],box[0]+box[2]',box)
        for k in range(int(box[0]),int(box[0]+box[2]),2):
            for l in range(int(box[1]),int(box[1]+box[3]),2):
                cnti+=1

                totali[0]+=CusTrackers.frame[l][k][0]
                totali[1]+=CusTrackers.frame[l][k][1]          
                totali[2]+=CusTrackers.frame[l][k][2]    

        recordi[0]=totali[0]/cnti
        recordi[1]=totali[1]/cnti       
        recordi[2]=totali[2]/cnti
        
        return recordi
        
    def checkPatternMatch(self,pattern,tocheck):
        if( abs(pattern[0]/pattern[1]-tocheck[0]/tocheck[1])<0.2 and 
           abs(pattern[1]/pattern[2]-tocheck[1]/tocheck[2])<0.2 and 
           abs(pattern[0]/pattern[2]-tocheck[0]/tocheck[2])<0.2 
          ) :
            return True
        else:
            return False
    def occulusionstart(self,listtopickone):  
        mindistancetomid=10000
        mindistanceno=-1
        for aa in range(len(listtopickone)): #find the box cloest to mid
            
           
            tracker1=CusTrackers.trackerlist[listtopickone[aa]] 
            distancetomid=math.sqrt((tracker1.boxmidpointx-CusTrackers.frame.shape[1]/2)*(tracker1.boxmidpointx-CusTrackers.frame.shape[1]/2)+
                        (tracker1.boxmidpointy-CusTrackers.frame.shape[0]/2)*(tracker1.boxmidpointy-CusTrackers.frame.shape[0]/2))
            if distancetomid<mindistancetomid:
                mindistancetomid=distancetomid
                mindistanceno=listtopickone[aa]
           

        listtopickone.remove(mindistanceno)
        for no in listtopickone:
            tracker=CusTrackers.trackerlist[no]
            
            thocculusionbox=self.occulusionedBox(tracker.no,tracker.box,tracker.pattern)
            CusTrackers.occulusionTakenList[tracker.no]=1
            
            
            CusTrackers.occulusionlist.append(thocculusionbox)           

        return mindistanceno,mindistancetomid
        #find the one at the front, by checking the mid 
    def deleteOcculusion(self,indexinlist):
        CusTrackers.occulusionTakenList[CusTrackers.occulusionlist[indexinlist].no]=0
        del CusTrackers.occulusionlist[indexinlist]
        
        
        
    def findNumbertoAssign(self):
     
        print('findNumbertoAssign: CusTrackers.numbertoAssignList',CusTrackers.numbertoAssignList)
        for i in range(len(CusTrackers.numbertoAssignList)):
           
            if CusTrackers.numbertoAssignList[i]==0 and  CusTrackers.occulusionTakenList[i]==0:
               # print(i)
                CusTrackers.numbertoAssignList[i]=1
                return i
     
        return -1
            
    def checkboxExists(self,box,typeno,threshold=0.5):  
        #typeno:1 tracker, 2,occubox
        foundmatch=0
        if typeno==1:
            for i in range(len(CusTrackers.trackerlist)):
                overlapsize=self.intersectionsize(box,CusTrackers.trackerlist[i].box)
                UnionSize=self.UnionSize(box,CusTrackers.trackerlist[i].box)

                if overlapsize/UnionSize>threshold :
                    foundmatch=1

                    return i
        elif typeno==2:
            for i in range(len(CusTrackers.occulusionlist)):
                overlapsize=self.intersectionsize(box,CusTrackers.occulusionlist[i].box)
                UnionSize=self.UnionSize(box,CusTrackers.occulusionlist[i].box)

                if overlapsize/UnionSize>threshold :
                    foundmatch=1

                    return i            

        
        return -1
                        
                        
    def addTracker(self,box,trackerno=-1,tracker_type=None,typef=None):
        print('box',box)
        if tracker_type is None:
            tracker_type=CusTrackers.tracker_type
        if typef is None:
            typef=CusTrackers.typef
        frame=CusTrackers.frame
        if trackerno==-1:
            no=self.findNumbertoAssign()
          #  print('trackerno==-1 no=self.findNumbertoAssign()',no)
        else:
            
            no=trackerno
          #  print('else no',no)
        
        #print('addTracker',no)
        #print(box)
        if(no==-1): #full
           # print('no==False')
            return False
       
       # print('box',box)
        tracker=self.perTracker(no,tuple(box),frame,tracker_type,typef)
       

        CusTrackers.trackerlist.append(tracker)
        
  
        CusTrackers.totalcount+=1
        return tracker

           # trackerdict['tracker']=tracker
       
      #  trackerlist2=[]
       # for tracker in CusTrackers.trackerlist:
         #   if(tracker.lostTrackCnt<10):
         #       trackerlist2.append(tracker)    
     #   CusTrackers.trackerlist=trackerlist2  #remove trackers with lostcnt>=10
        

    def rangebetweenpoints(self,p1,p2):
        return math.sqrt((p2[0]-p1[0])*(p2[0]-p1[0]))+((p2[1]-p1[1])*(p2[1]-p1[1]))

    
    def replaceTracker(self,tracker,standardtrackerbox):

        tracker_type=CusTrackers.tracker_type

        typef=CusTrackers.typef
        trackernew=self.perTracker(tracker.no,tuple(standardtrackerbox),CusTrackers.frame,tracker_type,typef)
                       
        trackernew.locationhistory=tracker.locationhistory
        trackernew.lostTrackCnt=0

        trackernew.detectedcnt=tracker.detectedcnt
        
        return trackernew
        
    
    def addBorders(self,listoflines): #listoflines = list of list of list, list of line (list of 2 points([x,y]))
        for line in listoflines:
            #findleft
            if line[0][0]<line[1][0]:
                leftpoint=line[0]
                rightpoint=line[1]
            else:
                rightpoint=line[0]
                leftpoint=line[1]     
         #   print('left,right',leftpoint,rightpoint)
            border=CusTrackers.perBorder(leftpoint,rightpoint)
            CusTrackers.borderList.append(border)

                
    def checkOverlap(self,contour,trackerbox,matheight=2880,matwidth=2880):
       # print('checkOverlap')
        ratio=0.5
        zeroMat1 = np.zeros((int(matheight*ratio), int(matwidth*ratio)),np.uint8)
        zeroMat2 = np.zeros((int(matheight*ratio), int(matwidth*ratio)),np.uint8)
        #dectcontour=contour.copy()
        dectcontour = copy.deepcopy(contour)
        trackercontour=[[trackerbox[0],trackerbox[1]],[trackerbox[0],trackerbox[1]+trackerbox[3]],
        [trackerbox[0]+trackerbox[2],trackerbox[1]+trackerbox[3]],[trackerbox[0]+trackerbox[2],trackerbox[1]]]
        #print(type(contour))
        
        for point in dectcontour:
           
            point[0]=int(point[0]*ratio)
            point[1]=int(point[1]*ratio)
          
        for point in trackercontour:
         
            point[0]=int(point[0]*ratio)
            point[1]=int(point[1]*ratio)    
        
        trackercontour=np.asarray(trackercontour,dtype=np.uint)
        dectcontour=np.asarray(dectcontour,dtype=np.uint)
        #print('checkOverlap','trackercontour',trackercontour)
        #print('checkOverlap','dectcontour',dectcontour)
        cv.fillConvexPoly(zeroMat1, trackercontour, 255)
        
        cv.fillConvexPoly(zeroMat2, dectcontour, 255)
        mat3=cv.bitwise_and(zeroMat1, zeroMat2)
        nzCount = cv.countNonZero(mat3);
        #print(nzCount)
        #print(trackerbox[2]*trackerbox[3])
        overlapratio=nzCount/(trackerbox[2]*trackerbox[3]*ratio*ratio)
        
        
        return overlapratio
    def getDistanceBetweenPoints(self,p1,p2):
        distance=math.sqrt(math.pow((p1[0]-p2[0]),2)+math.pow((p1[1]-p2[1]),2))
        return distance
    def findNearestContour(self,box):
        #print(box)
        boxmidpoint=[[box[0]+box[2]/2],[box[1]+box[3]/2]]
        minrange=99999
        nearestindex=999
        if len(CusTrackers.detectContourList)==0:
            return False,False
        for i in range(len(CusTrackers.detectContourList)):
            
            contour=CusTrackers.detectContourList[i].contour
            #print(CusTrackers.detectContourList[i].contour)
            contourmidpointx=(contour[0][0]+contour[1][0]+contour[2][0]+contour[3][0])/4
            contourmidpointy=(contour[0][1]+contour[1][1]+contour[2][1]+contour[3][1])/4
            midrange=self.getDistanceBetweenPoints([contourmidpointx,contourmidpointy],[boxmidpoint[0],boxmidpoint[1]])
            #print('midrange',midrange,minrange)
            if(midrange<minrange):
                minrange=midrange
                nearestindex=i

        return CusTrackers.detectContourList[nearestindex],nearestindex
            
    def getTrackerBoxfromContour(self,contour):
        #print('contour',contour)
        
        tophalfcontourleftbtmx=int((contour[0][0]+contour[1][0])/2)
        tophalfcontourleftbtmy=int((contour[0][1]+contour[1][1])/2)
        tophalfcontourrightbtmx=int((contour[3][0]+contour[2][0])/2)
        tophalfcontourrightbtmy=int((contour[3][1]+contour[2][1])/2)
        contourmidx=(contour[0][0]+contour[1][0]+contour[2][0]+contour[3][0])/4
        contourmidy=(contour[0][1]+contour[1] [1]+contour[2][1]+contour[3][1])/4
        
        tophalfcontour=[[contour[0][0],contour[0][1]],[tophalfcontourleftbtmx,tophalfcontourleftbtmy],
                        [tophalfcontourrightbtmx,tophalfcontourrightbtmy],[contour[3][0],contour[3][1]]]
        #print('tophalfcontour',tophalfcontour) 
        topmidx=(tophalfcontour[0][0]+tophalfcontour[1][0]+tophalfcontour[2][0]+tophalfcontour[3][0])/4
        topmidy=(tophalfcontour[0][1]+tophalfcontour[1][1]+tophalfcontour[2][1]+tophalfcontour[3][1])/4
        boxmidx=(topmidx+contourmidx)/2
        boxmidy=(topmidy+contourmidy)/2
        contourheight=(self.getDistanceBetweenPoints(contour[0],contour[1])+self.getDistanceBetweenPoints(contour[2],contour[3]))/2
        contourwidth=(self.getDistanceBetweenPoints(contour[0],contour[3])+self.getDistanceBetweenPoints(contour[1],contour[2]))/2
        halfcontourheight=contourheight/2
        newboxsidelength=int((contourwidth)/1.5)
        newtrackerbox=list(map(int,[(boxmidx-newboxsidelength/2),(boxmidy-newboxsidelength/2),newboxsidelength,newboxsidelength]))
        return newtrackerbox
    def intersectionsize(self,box1, box2):
        ax1=box1[0]
        ax2=box1[0]+box1[2]
        ay1=box1[1]
        ay2=box1[1]+box1[3]
        
        bx1=box2[0]
        bx2=box2[0]+box2[2]
        by1=box2[1]
        by2=box2[1]+box2[3]        
        
        x1 = max(min(ax1, ax2), min(bx1, bx2))
        y1 = max(min(ay1, ay2), min(by1, by2))
        x2 = min(max(ax1, ax2), max(bx1, bx2))
        y2 = min(max(ay1, ay2), max(by1, by2))
        
        if x1<x2 and y1<y2:
            return (x2-x1)*(y2-y1)
        else:
            return 0

    def UnionSize(self,box1, box2):
        ax1=box1[0]
        ax2=box1[0]+box1[2]
        ay1=box1[1]
        ay2=box1[1]+box1[3]

        bx1=box2[0]
        bx2=box2[0]+box2[2]
        by1=box2[1]
        by2=box2[1]+box2[3]        

        x1 = max(min(ax1, ax2), min(bx1, bx2))
        y1 = max(min(ay1, ay2), min(by1, by2))
        x2 = min(max(ax1, ax2), max(bx1, bx2))
        y2 = min(max(ay1, ay2), max(by1, by2))

        size1=(ax2-ax1)*(ay2-ay1)
        size2=(bx2-bx1)*(by2-by1)
        Unionsize=size1+size2
        if x1<x2 and y1<y2:
            return Unionsize-(x2-x1)*(y2-y1)
        else:
            return Unionsize  
        
    def z2polar(self,z):
        print('z',z)
        return ( abs(z), angle(z) )
    
    def checkPolarNear(self,box1,box2):
        midpoint1x=(box1[0]+box1[0]+box1[2])/2
        midpoint1y=(box1[1]+box1[1]+box1[3])/2
        
        midpoint2x=(box2[0]+box2[0]+box2[2])/2
        midpoint2y=(box2[1]+box2[1]+box2[3])/2
        frame=CusTrackers.frame
        framemidx=int(frame.shape[1]/2)
        framemidy=int(frame.shape[0]/2)
        
        box1toframex=int(midpoint1x-framemidx)
        box1toframey=int(midpoint1y-framemidy)
        box2toframex=int(midpoint2x-framemidx)
        box2toframey=int(midpoint2y-framemidy)
        
        r1,theta1=self.z2polar(complex(box1toframex,box1toframey) )
        r2,theta2=self.z2polar(complex(box2toframex,box2toframey) )
        
        print('midpoint1y,framemidy,box1toframex,box1toframey,box2toframex,box2toframey,box1,box2,r1,r2,theta1,theta2',
              midpoint1y,framemidy,box1toframex,box1toframey,box2toframex,box2toframey,box1,box2,r1,r2,theta1,theta2)
        if r1/r2>0.8 and r1/r2<1.2 and abs(theta1-theta2)<0.25:
            return True
        else:
            return False
        
   
    
   # def delTracker(self,tracker):
                
    #def syncWithDetect(self,detectbox):
        
    #def check_roi

btn_down = False

def get_points(im):
    # Set up data to send to mouse handler
    data = {}
    data['im'] = im.copy()
    data['lines'] = []

    # Set the callback function for any mouse event
    cv.imshow("Image", im)
    cv.setMouseCallback("Image", mouse_handler, data)
    cv.waitKey(0)
    cv.destroyAllWindows()
    # Convert array to np.array in shape n,2,2
    points = np.uint16(data['lines'])

    return points, data['im']

def mouse_handler(event, x, y, flags, data):
    global btn_down

    if event == cv.EVENT_LBUTTONUP and btn_down:
        #if you release the button, finish the line
        btn_down = False
        data['lines'][0].append((x, y)) #append the seconf point
        cv.circle(data['im'], (x, y), 3, (0, 0, 255),5)
        cv.line(data['im'], data['lines'][0][0], data['lines'][0][1], (0,0,255), 2)
        cv.imshow("Image", data['im'])

    elif event == cv.EVENT_MOUSEMOVE and btn_down:
        #thi is just for a ine visualization
        image = data['im'].copy()
        cv.line(image, data['lines'][0][0], (x, y), (0,0,255), 2)
        cv.imshow("Image", image)

    elif event == cv.EVENT_LBUTTONDOWN and len(data['lines']) < 2:
        btn_down = True
        data['lines'].insert(0,[(x, y)]) #prepend the point
        cv.circle(data['im'], (x, y), 3, (0, 0, 255), 5, 16)
        cv.imshow("Image", data['im'])

# Running the code
"""
img = np.zeros((1000, 1000,3))
pts, final_image = get_points(img)
#cv.imshow('Image', final_image)
print (pts)
#cv.waitKey(0)
"""

"\nimg = np.zeros((1000, 1000,3))\npts, final_image = get_points(img)\n#cv.imshow('Image', final_image)\nprint (pts)\n#cv.waitKey(0)\n"

In [3]:
def getPerpCoord(aX, aY, bX, bY, length): #a line perpendicular to line(a to b) and with b in the mid 
    vX = bX-aX
    vY = bY-aY
    #print(str(vX)+" "+str(vY))

    mag = math.sqrt(vX*vX + vY*vY)
    vX = vX / mag
    vY = vY / mag
    temp = vX
    vX = 0-vY
    vY = temp
    cX = bX + vX * length
    cY = bY + vY * length
    dX = bX - vX * length
    dY = bY - vY * length
    return int(cX), int(cY), int(dX), int(dY)   

print(getPerpCoord(0,0,0,1,1))

(-1, 1, 1, 1)


In [4]:



# config_name = 'config/mobilenetv2_300_person_wework.json'
# config_name = 'config/mobilenetv2_300_person_wework_anchor2.json'
config_name = 'config/mobilenetv2_300_person_wework_480.json'

# config_name = 'config/mobilenetv2_300_person.json'
# config_name = 'config/mobilenetv2_96_face.json'

sys.path.insert(0, 'net')
with open(config_name, 'r') as config_file:
    config_args_dict = json.load(config_file)
    config_args = edict(config_args_dict)
ssd_net = __import__(config_args.model_file)



def layer_params(layers_shapes, anchor_scales, anchor_ratios, extra_anchor_scales):
    feature_num = []
    feature_depth = []
    anchor_num = 0
    for id, layer in enumerate(layers_shapes):
        feature_num.append(layer[0] * layer[1])
        feature_depth.append(len(anchor_scales[id]) *len(anchor_ratios[id])  + len(extra_anchor_scales[id]))
        anchor_num += (layer[0] * layer[1])* feature_depth[-1]

    return anchor_num, feature_num, feature_depth
    

# scaffold related configuration
tf.app.flags.DEFINE_integer(
    'num_classes', 2, 'Number of classes to use in the dataset.')
# model related configuration
tf.app.flags.DEFINE_integer(
    'train_image_size', config_args.input_size,
    'The size of the input image for the model to use.')
tf.app.flags.DEFINE_string(
    'data_format', 'channels_first', # 'channels_first' or 'channels_last'
    'A flag to override the data format used in the model. channels_first '
    'provides a performance boost on GPU but is not always compatible '
    'with CPU. If left unspecified, the data format will be chosen '
    'automatically based on whether TensorFlow was built for CPU or GPU.')
tf.app.flags.DEFINE_float(
    'select_threshold', 0.2, 'Class-specific confidence score threshold for selecting a box.')
tf.app.flags.DEFINE_float(
    'min_size', 0.1, 'The min size of bboxes to keep.')
tf.app.flags.DEFINE_float(
    'nms_threshold', 0.45, 'Matching threshold in NMS algorithm.')
tf.app.flags.DEFINE_integer(
    'nms_topk', 20, 'Number of total object to keep after NMS.')
tf.app.flags.DEFINE_integer(
    'keep_topk', 100, 'Number of total object to keep for each image before nms.')
# checkpoint related configuration
tf.app.flags.DEFINE_string(
    'checkpoint_path', config_args.logdir,
    'The path to a checkpoint from which to fine-tune.')
tf.app.flags.DEFINE_string(
    'model_scope', 'ssd300',
    'Model scope name used to replace the name_scope in checkpoint.')

FLAGS = tf.app.flags.FLAGS
#CUDA_VISIBLE_DEVICES

def get_checkpoint():
    if tf.gfile.IsDirectory(FLAGS.checkpoint_path):
        checkpoint_path = tf.train.latest_checkpoint(FLAGS.checkpoint_path)
    else:
        checkpoint_path = FLAGS.checkpoint_path

    return checkpoint_path

def select_bboxes(scores_pred, bboxes_pred, num_classes, select_threshold):
    selected_bboxes = {}
    selected_scores = {}
    with tf.name_scope('select_bboxes', values=[scores_pred, bboxes_pred]):
        for class_ind in range(1, num_classes):
            class_scores = scores_pred[:, class_ind]
            # select_mask = class_scores > select_threshold
            # select_mask = tf.cast(select_mask, tf.float32)
            select_mask = tf.where(class_scores > select_threshold, tf.ones_like(class_scores), tf.zeros_like(class_scores))
            selected_bboxes[class_ind] = tf.multiply(bboxes_pred, tf.expand_dims(select_mask, axis=-1))
            selected_scores[class_ind] = tf.multiply(class_scores, select_mask)
        print('\n==\nselected_bboxes = {}'.format(selected_bboxes))
        print('\n==\nselected_scores = {}'.format(selected_scores))
    return selected_bboxes, selected_scores

def clip_bboxes(cy, cx, h, w, name):
    with tf.name_scope(name, 'clip_bboxes', [cy, cx, h, w]):
        cy = tf.maximum(cy, 0.)
        cx = tf.maximum(cx, 0.)
        cy = tf.minimum(cy, 1.)
        cx = tf.minimum(cx, 1.)
        h = tf.maximum(h, 0.)
        w = tf.maximum(w, 0.)
        h = tf.minimum(h, 1.)
        w = tf.minimum(w, 1.)

        return cy, cx, h, w

def filter_bboxes(scores_pred, ymin, xmin, ymax, xmax, min_size, name):
    with tf.name_scope(name, 'filter_bboxes', [scores_pred, ymin, xmin, ymax, xmax]):
        width = xmax - xmin
        height = ymax - ymin

        filter_mask = tf.logical_and(width > min_size, height > min_size)

        filter_mask = tf.cast(filter_mask, tf.float32)
        return tf.multiply(ymin, filter_mask), tf.multiply(xmin, filter_mask), \
                tf.multiply(ymax, filter_mask), tf.multiply(xmax, filter_mask), tf.multiply(scores_pred, filter_mask)

def sort_bboxes(scores_pred, cy, cx, h, w, keep_topk, name):
    with tf.name_scope(name, 'sort_bboxes', [scores_pred, cy, cx, h, w]):
        cur_bboxes = tf.shape(scores_pred)[0]
        scores, idxes = tf.nn.top_k(scores_pred, k=tf.minimum(keep_topk, cur_bboxes), sorted=True)

        cy, cx, h, w = tf.gather(cy, idxes), tf.gather(cx, idxes), tf.gather(h, idxes), tf.gather(w, idxes)

        paddings_scores = tf.expand_dims(tf.stack([0, tf.maximum(keep_topk-cur_bboxes, 0)], axis=0), axis=0)

        return tf.pad(cy, paddings_scores, "CONSTANT"), tf.pad(cx, paddings_scores, "CONSTANT"),\
                tf.pad(h, paddings_scores, "CONSTANT"), tf.pad(w, paddings_scores, "CONSTANT"),\
                tf.pad(scores, paddings_scores, "CONSTANT")

def nms_bboxes(scores_pred, bboxes_pred, nms_topk, nms_threshold, name):
    with tf.name_scope(name, 'nms_bboxes', [scores_pred, bboxes_pred]):
        idxes = tf.image.non_max_suppression(bboxes_pred, scores_pred, nms_topk, nms_threshold)
        return tf.gather(scores_pred, idxes), tf.gather(bboxes_pred, idxes)

def center2point( center_y, center_x, height, width):
    # print(center_y.shape)
    angle = np.arctan((center_y - 0.5)/( center_x - 0.5 + 1e-20))
    pi = math.pi
    angle = np.where(np.less_equal(center_x, 0.5) , pi - angle, -angle)
    angle = np.where(np.less_equal(angle, 0.0), pi + pi + angle, angle)
    # print(angle.shape)
    rotation_matrix = np.stack([math.cos(angle), -math.sin(angle),  
                        math.sin(angle),math.cos(angle)], axis=0)
    rotation_matrix = np.reshape(rotation_matrix, (2, 2))
    height, width = width, height 
    points = np.stack([[ -width / 2,  -height / 2], [ width / 2,  -height / 2], [ width / 2,  height / 2], [ -width / 2,  height / 2] ], axis=0)
    points = np.matmul(points, rotation_matrix) + [center_x, center_y]
    return points

def fill_poly(points, size):
    img = np.zeros((int(size),int(size)))
    points = np.int32(points * size)
    # points = np.stack([[int(pt[0] * size), int(pt[1] * size)] for pt in points], axis = 0)
    cv.fillPoly(img, [points], 1)
    return img

def get_box_mask( center_y, center_x, height, width):
    center_y, center_x, height, width = [np.reshape(b, [-1]) for b in [center_y, center_x, height, width]]
    input_value = np.stack([center_y, center_x, height, width], axis = 1)
    points = [center2point(cy, cx, h, w ) for cy, cx, h, w in zip(center_y, center_x, height, width)]
    image = [fill_poly(pts, 96.) for pts in points]
    image = np.stack(image, axis=0)
    return image

# def compute_iou(mask1, mask2):
#     intersection = np.sum(mask1*mask2)
#     if intersection > 0.0:
#         return intersection / (sum(mask1) + sum(mask2) - intersection)
#     else:
#         return 0.0

def compute_iou(box, boxes, box_area, boxes_area):
    dist = np.reshape(np.square(box[0] - boxes[:,0] ) + np.square(box[1] - boxes[:,1]), [-1,1])
    size = np.reshape(np.square(np.maximum(np.maximum(box[2], box[3]), np.maximum(boxes[:,2], boxes[:,3]))), [-1,1])
    box_area = np.expand_dims(np.expand_dims(box_area, 0), 0)
    boxes_area = np.expand_dims(boxes_area, 1)
    cond = dist < size
    inter_area = np.sum(np.multiply(box_area, boxes_area), (2,3))
    # print('inter_area = {}, cond = {}'.format(inter_area.shape, cond.shape))
    # input()
    inter_vol = np.where(dist< size, np.sum(np.multiply(box_area, boxes_area), (2,3)), 0)
    area1 = np.sum(box_area, (2,3))
    area2 = np.sum(boxes_area, (2,3))
    union_vol = area1 + area2 - inter_vol
    return np.where(np.equal(union_vol, 0.0), np.zeros_like(inter_vol), inter_vol/ union_vol)



def non_max_suppression(scores_pred, bboxes_pred, nms_topk, nms_threshold):
    assert bboxes_pred.shape[0] == scores_pred.shape[0]
    cy = bboxes_pred[:, 0]
    cx = bboxes_pred[:, 1]
    h = bboxes_pred[:, 2]
    w = bboxes_pred[:, 3]
    masks = get_box_mask(cy, cx, h, w)
    # # box coordinate ranges are inclusive-inclusive
    # areas = (ys2 - ys1) * (xs2 - xs1)
    scores_indexes = scores_pred.argsort().tolist()
    boxes_keep_index = []
    while len(scores_indexes):
        index = scores_indexes.pop()
        boxes_keep_index.append(index)
        if not len(scores_indexes):
            break
        ious = compute_iou(bboxes_pred[index], bboxes_pred[scores_indexes], masks[index],
                           masks[scores_indexes])
        filtered_indexes = set((ious > nms_threshold).nonzero()[0])
        # if there are no more scores_index
        # then we should pop it
        scores_indexes = [
            v for (i, v) in enumerate(scores_indexes)
            if i not in filtered_indexes
        ]
    
    if len(boxes_keep_index) > nms_topk:
      boxes_keep_index = boxes_keep_index[:nms_topk]
    return np.array(boxes_keep_index)




def parse_by_class(cls_pred, bboxes_pred, num_classes, select_threshold, min_size, keep_topk, nms_topk, nms_threshold):
    with tf.name_scope('select_bboxes', values=[cls_pred, bboxes_pred]):
        scores_pred = tf.nn.softmax(cls_pred)
        selected_bboxes, selected_scores = select_bboxes(scores_pred, bboxes_pred, num_classes, select_threshold)

        for class_ind in range(1, num_classes):
            cy, cx, h, w = tf.unstack(selected_bboxes[class_ind], 4, axis=-1)
            ymin, xmin, ymax, xmax = tf.unstack(selected_bboxes[class_ind], 4, axis=-1)
            #ymin, xmin, ymax, xmax = tf.squeeze(ymin), tf.squeeze(xmin), tf.squeeze(ymax), tf.squeeze(xmax)
            cy, cx, h, w = clip_bboxes(cy, cx, h, w, 'clip_bboxes_{}'.format(class_ind))
            # ymin, xmin, ymax, xmax, selected_scores[class_ind] = filter_bboxes(selected_scores[class_ind],
            #                                     ymin, xmin, ymax, xmax, min_size, 'filter_bboxes_{}'.format(class_ind))
            cy, cx, h, w , selected_scores[class_ind] = sort_bboxes(selected_scores[class_ind],
                                                cy, cx, h, w , keep_topk, 'sort_bboxes_{}'.format(class_ind))
            selected_bboxes[class_ind] = tf.stack([cy, cx, h, w ], axis=-1)
            # selected_scores[class_ind], selected_bboxes[class_ind] = nms_bboxes(selected_scores[class_ind], selected_bboxes[class_ind], nms_topk, nms_threshold, 'nms_bboxes_{}'.format(class_ind))

        return selected_bboxes, selected_scores

def preprocessing(np_image, input_format, output_format, data_format, bias):
    # _R_MEAN = 123.68
    # _G_MEAN = 116.78
    # _B_MEAN = 103.94
    image = cv.resize(np_image, (config_args.input_size, config_args.input_size))
    image = image.astype(float)
    
    if input_format == 'bgr':
      test_image = np.concatenate([np.expand_dims(image[:,:,2], axis = -1),
                                  np.expand_dims(image[:,:,1], axis = -1), 
                                  np.expand_dims(image[:,:,0], axis = -1)], axis = 2)
    else:
      test_image = image                                          
    test_image = test_image - np.expand_dims(np.expand_dims(bias, axis = 0), axis = 0)
    if output_format == 'bgr':
      output_image = np.concatenate([np.expand_dims(test_image[:,:,2], axis = -1),
                                    np.expand_dims(test_image[:,:,1], axis = -1), 
                                    np.expand_dims(test_image[:,:,0], axis = -1)], axis = 2)
    else:
      output_image = test_image  
    if data_format == 'channels_first':
        output_image = np.transpose(output_image, (2, 0, 1))
    output_image = output_image * 2.0 / 255.0
    return np.expand_dims(output_image, axis = 0)


def main(_):
    make_npy('./priorbox_data.npy', config_args)
    with tf.Graph().as_default():
        out_shape = [FLAGS.train_image_size] * 2

        ''' 
        # tf not do preprocessing 
        image_input = tf.placeholder(tf.uint8, shape=(None, None, 3), name = 'input_image')
        shape_input = tf.shape(image_input)[:-1]
        #shape_input = tf.placeholder(tf.int32, shape=(2,), name = 'input_shape')
        features = ssd_preprocessing.preprocess_for_eval(image_input, out_shape, data_format=FLAGS.data_format, output_rgb=False)
        features = tf.expand_dims(features, axis=0)
        '''
        if FLAGS.data_format == 'channels_first':
            features = tf.placeholder(tf.float32, shape=(1, 3, config_args.input_size, config_args.input_size), name = 'input_image')
        else:
            features = tf.placeholder(tf.float32, shape=(1, config_args.input_size, config_args.input_size, 3), name = 'input_image')

        anchor_creator = anchor_manipulator.AnchorCreator(out_shape,
            layers_shapes = config_args.layers_shapes,
            anchor_scales = config_args.anchor_scales,
            extra_anchor_scales = config_args.extra_anchor_scales,
            anchor_ratios = config_args.anchor_ratios,
            layer_steps = config_args.layer_steps)

        all_anchors, all_num_anchors_depth, all_num_anchors_spatial = anchor_creator.get_all_anchors()
        
        anchor_encoder_decoder = anchor_manipulator.AnchorEncoder(allowed_borders = [1.0] * config_args.layer_num,
                                                        positive_threshold = None,
                                                        ignore_threshold = None,
                                                        prior_scaling=[0.1, 0.1, 0.1, 0.1])
        decode_fn = lambda pred : anchor_encoder_decoder.ext_decode_all_anchors(pred, all_anchors, all_num_anchors_depth, all_num_anchors_spatial)

        with tf.variable_scope(FLAGS.model_scope, default_name=None, values=[features], reuse=tf.AUTO_REUSE):
            backbone = ssd_net.MobileNetV2(FLAGS.data_format)
            feature_layers = backbone.forward(features, training=False)
            location_pred, cls_pred = ssd_net.multibox_head(feature_layers, FLAGS.num_classes, all_num_anchors_depth, data_format=FLAGS.data_format)
            if FLAGS.data_format == 'channels_first':
                cls_pred = [tf.transpose(pred, [0, 2, 3, 1]) for pred in cls_pred]
                location_pred = [tf.transpose(pred, [0, 2, 3, 1]) for pred in location_pred]
                        
            anchor_num, feature_num, feature_depth = layer_params(config_args.layers_shapes, config_args.anchor_scales, config_args.anchor_ratios, config_args.extra_anchor_scales)

            cls_pred = [tf.reshape(pred, [1, -1, FLAGS.num_classes, 1])  for pred in cls_pred]
            location_pred = [tf.reshape(pred, [1, -1, 4, 1]) for pred in location_pred]            
            cls_pred = tf.concat(cls_pred, axis=1, name = 'concat_cls')
            location_pred = tf.concat(location_pred, axis=1, name = 'concat_loc')


        with tf.device('/cpu:0'):
            location_pred_pos = tf.reshape(location_pred, tf.shape(location_pred)[1:-1])
            cls_pred_pos = tf.reshape(cls_pred, tf.shape(cls_pred)[1:-1])
            bboxes_pred = decode_fn(location_pred_pos)
            bboxes_pred = tf.concat(bboxes_pred, axis=0)
            selected_bboxes, selected_scores = parse_by_class(cls_pred_pos, bboxes_pred,
                                                            FLAGS.num_classes, FLAGS.select_threshold, FLAGS.min_size,
                                                            FLAGS.keep_topk, FLAGS.nms_topk, FLAGS.nms_threshold)
            labels_list = []
            scores_list = []
            bboxes_list = []
            for k, v in selected_scores.items():
                labels_list.append(tf.ones_like(v, tf.int32) * k)
                scores_list.append(v)
                bboxes_list.append(selected_bboxes[k])
            all_labels = tf.concat(labels_list, axis=0)
            all_scores = tf.concat(scores_list, axis=0)
            all_bboxes = tf.concat(bboxes_list, axis=0)
            all_pres = tf.concat([tf.expand_dims(all_scores, axis = 1), all_bboxes], axis=1)

        
        saver = tf.train.Saver()
        gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.7)
        with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options,log_device_placement=False)) as sess:
            init = tf.global_variables_initializer()
            sess.run(init)
            checkpoint_path = get_checkpoint()
            saver.restore(sess, checkpoint_path)
            tf.summary.FileWriter('./freezelogs', sess.graph)

            ''' 
            # show nodmes
            node_names = [n.name for n in tf.get_default_graph().as_graph_def().node]
            for name in node_names:
                print(name)
            '''

            # freeze model exclude preprocessing and postporcessing
            priorbox_np = np.reshape(np.load('./priorbox_data.npy'), [-1,1,anchor_num*4,1])
            scale_np = np.reshape(np.matmul(np.ones((anchor_num,1)), [[0.1, 0.1, 0.1, 0.1]]), [-1,1,anchor_num*4,1])
            priorbox_data = tf.concat([tf.constant(priorbox_np, dtype=tf.float32), tf.constant(scale_np, dtype=tf.float32)], axis = 1)
            priorbox_data = tf.reshape(priorbox_data, [-1, 2 , anchor_num*4,1], name = 'priorbox_data' )
            # cls_pred = tf.nn.softmax(cls_pred, axis=2)
            loc_data = tf.reshape(location_pred, [-1, anchor_num*4, 1, 1], name = 'loc_data')
            conf_data = tf.reshape(cls_pred, [-1, anchor_num*2, 1, 1], name = 'conf_data')
            tf.train.write_graph(sess.graph_def, 'model', 'ssd_model.pb')
            freeze_graph.freeze_graph('model/ssd_model.pb',
                      '', False, checkpoint_path  , 'loc_data,conf_data, priorbox_data', 'save/restore_all', 'save/Const:0',
                       'ssdface_freeze/' + config_args.freeze_file + '.pb', False, "")
            print("FREEZE done")
 

            # freeze model exclude preprocessing 
            # outputs = tf.identity(all_pres, name='outputs')
            # tf.train.write_graph(sess.graph_def, 'model', 'face_ssd_model_nonpreprocess.pb')
            # freeze_graph.freeze_graph('model/face_ssd_model_nonpreprocess.pb',
            #           '', False, checkpoint_path  , 'outputs', 'save/restore_all', 'save/Const:0',
            #            'ssdface_freeze/face_ssd_model_fz.pb', False, "")
            # print("FREEZE done")


            def read_labelme(json_path):
                if not os.path.isfile(json_path) :
                    return []
                if not '.json' in json_path:
                    return False
                with open(json_path) as json_file:
                    data = json.load(json_file)
                all_boxes = []
                if 'shapes' in data:
                    for obj in data['shapes']:
                        box = obj['points']
                        if len(box):
                            box = [[box[0][0], box[0][1]], [box[1][0],box[1][1]], [box[2][0],box[2][1]], [box[3][0],box[3][1]]]
                        # box = (map(int, box))
                        all_boxes.append(box)
                return all_boxes


            def test_img(filename):
                np_image = cv.imread(filename)
                # np_image = np_image[:,:np_image.shape[1],:]
                # np_image = np.ones((1,3,96,96), dtype=np.float32) * 0.5
                show_image = np_image.copy()
                np_image = preprocessing(np_image, input_format = 'bgr', output_format = 'rgb', data_format = FLAGS.data_format, bias = config_args.bias)

                start = time.time()
                # all_pres_, feature_layers_, all_anchors_ = sess.run(
                #     [all_pres, feature_layers, all_anchors], 
                #     feed_dict = {features : np_image}) 
                labels_, scores_, bboxes_ = sess.run(
                    [all_labels, all_scores, all_bboxes], 
                    feed_dict = {features : np_image})
                idxes = non_max_suppression(scores_, bboxes_, 20, 0.45)
                scores = np.take(scores_, idxes)
                bboxes = np.take(bboxes_, idxes, axis = 0)
                labels = np.take(labels_, idxes)


                end = time.time()
                #!!!

                img_to_draw = draw_toolbox.rotate_bboxes_draw_on_img(show_image, labels, scores, bboxes, thickness=5, xy_order = 'xy', color=(155,155,255))
                # img_to_draw = draw_toolbox.bboxes_draw_on_img(show_image, labels_, scores_, bboxes_, thickness=5, xy_order='yx')
                
                #bboxes 
                
                polygon = read_labelme(filename[:-4] + '.json')
                # print(polygon)
                for poly in polygon:
                    poly = np.stack(poly, axis=0).astype(int)
                    cv.polylines(img_to_draw,[poly.astype(int)],True,(0,255,255),2)

                # cv.putText(img_to_draw, "tracker = {:.0f} ms".format((end - start)/10 * 1000), (30,60), 2, 0.7, (255,55,155))
                cv.imwrite('./demo/' + filename.split('/')[-1], img_to_draw)
                # cv.imshow('./demo/test_out.jpg', img_to_draw[:,:,:])
                # print('\nrun time = {:.0f}ms'.format((end - start)/10 * 1000))
                

                cv.waitKey()

            # test image
            # test_folder = '/home/scchiu/Data/HABBOF/Meeting1' 
            # file_list = os.listdir(test_folder)
            # for filename in file_list:
            #     if 'jpg' in filename:
            #         test_img(os.path.join(test_folder, filename))
            # test_img('./demo/test.jpg')
            


            # test video/webcam



            cnt = 0
            time2 = 0
            start = 0
            cap = cv.VideoCapture('video010.mp4')

            #fourcc = cv.VideoWriter_fourcc(*'XVID')
           # out = cv.VideoWriter('./video010' + '_output.avi', fourcc, 15.0, (1024, 1024))
            out = cv.VideoWriter('./video010' + '_output.avi',cv.VideoWriter_fourcc('M','J','P','G'), 15, (1024, 1024))
            a = True
            ret, frame = cap.read() 
            custrackers=CusTrackers()
            gotborders=0
            while(a):
                cnt += 1
                
                ret, frame = cap.read() 

                labels_, scores_, bboxes_ = sess.run( [all_labels, all_scores, all_bboxes], 
                    feed_dict = {features : preprocessing(frame, input_format = 'bgr', output_format = 'rgb', data_format = FLAGS.data_format, bias = config_args.bias)})
                idxes = non_max_suppression(scores_, bboxes_, 20, 0.45)
                scores = np.take(scores_, idxes)
                bboxes = np.take(bboxes_, idxes, axis = 0)
                labels = np.take(labels_, idxes)


                end = time.time()
                img_to_draw = cv.resize(frame, (600,600))
                custrackers.updateframe(img_to_draw)
                custrackers.clearDetectionContours()
               
                if gotborders==0:
                    pts, _ = get_points(img_to_draw)
                    print('pts',pts)
                    custrackers.addBorders(pts)
                    gotborders=1
                img_to_draw,pointslist = draw_toolbox.rotate_bboxes_nodraw_on_img(img_to_draw, labels, scores, bboxes, thickness=2, xy_order = 'xy', color=(155,155,255))
                for points in pointslist:
                    temp1=copy.copy(points[1])
                    points[1]=points[3]
                    points[3]=temp1
                
              #  print('pointslist',pointslist)
                for points in pointslist:

                    person_contour=(points[0],points[1],points[2],points[3])
                    custrackers.addDetectionContours(person_contour)
                    t1 = time.time()
   
        
                custrackers.updateTrackers()
                t_dewarp = time.time() - t1
                print('updateTrackers time {:d} ms'.format(int(t_dewarp * 1000)))  
                for i in range(len(custrackers.trackerlist)):
                    bbox=custrackers.trackerlist[i].box
                    no=custrackers.trackerlist[i].no
                    detectedcnt=custrackers.trackerlist[i].detectedcnt
                    lostTrackCnt=custrackers.trackerlist[i].lostTrackCnt
                  #  print('no,box,detectedcnt,lostTrackCnt',no,bbox,detectedcnt,lostTrackCnt)

                t1 = time.time()
                custrackers.adjustTrackersbyDetection()
                t_dewarp = time.time() - t1
                print('adjustTrackersbyDetection time {:d} ms'.format(int(t_dewarp * 1000)))  
                #print('adjustTrackersbyDetection()',len(custrackers.trackerlist))
                #print('len(custrackers.detectContourList)',len(custrackers.detectContourList))
                #for detection in custrackers.detectContourList:
                #    print(detection.trackerno)

                for i in range(len(custrackers.trackerlist)):
                    bbox=custrackers.trackerlist[i].box
                    no=custrackers.trackerlist[i].no
                    detectedcnt=custrackers.trackerlist[i].detectedcnt
                    lostTrackCnt=custrackers.trackerlist[i].lostTrackCnt

                 #   print('no,box,detectedcnt,lostTrackCnt',no,bbox,detectedcnt,lostTrackCnt)

                t1 = time.time()
                custrackers.recordPatternofNear()
                custrackers.retireTrackersAndOcculusion() 
                t_dewarp = time.time() - t1
                print('retireTrackers time {:d} ms'.format(int(t_dewarp * 1000)))
                for i in range(len(custrackers.trackerlist)):
                    bbox=custrackers.trackerlist[i].box
                    no=custrackers.trackerlist[i].no
                    detectedcnt=custrackers.trackerlist[i].detectedcnt
                    lostTrackCnt=custrackers.trackerlist[i].lostTrackCnt
                   # print('no,box,detectedcnt,lostTrackCnt',no,bbox,detectedcnt,lostTrackCnt)

                t1 = time.time()

                custrackers.addTrackersbyDetection()
                t_dewarp = time.time() - t1
                print('addTrackersbyDetection time {:d} ms'.format(int(t_dewarp * 1000)))  
                for i in range(len(custrackers.trackerlist)):
                    bbox=custrackers.trackerlist[i].box
                    no=custrackers.trackerlist[i].no
                    detectedcnt=custrackers.trackerlist[i].detectedcnt
                    lostTrackCnt=custrackers.trackerlist[i].lostTrackCnt
                  #  print('no,box,detectedcnt,lostTrackCnt',no,bbox,detectedcnt,lostTrackCnt)



                t1 = time.time()
                custrackers.nms()
                t_dewarp = time.time() - t1


                print('nms time {:d} ms'.format(int(t_dewarp * 1000)))  
                #print('addTrackersbyDetection()',len(custrackers.trackerlist))
                
                custrackers.updateBorders()
                

                #print('retireTrackers()',len(custrackers.trackerlist))
                img_to_draw,pointslist = draw_toolbox.rotate_bboxes_draw_on_img(img_to_draw, labels, scores, bboxes, thickness=2, xy_order = 'xy', color=(155,155,255))
                for i in range(len(custrackers.trackerlist)):
                    bbox=custrackers.trackerlist[i].box
                    no=custrackers.trackerlist[i].no
                    lostcnt=custrackers.trackerlist[i].no
                    history=custrackers.trackerlist[i].locationhistory
                #    print('no,box',no,bbox)
                    if ret:
                        # Tracking success
                        p1 = (int(bbox[0]), int(bbox[1]))
                        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                        cv.rectangle(img_to_draw, p1, p2, (255,0,0), 5, 1)           
                        cv.putText(img_to_draw, str(no), (int(bbox[0]+bbox[2]/2), int(bbox[1]+bbox[3]/2)), cv.FONT_HERSHEY_SIMPLEX,
          2, (0, 255, 255), 3, cv.LINE_AA)
                        for point in history :
                            cv.circle(img_to_draw,tuple(point), 5, (255, 0, 255), -1)
                for border in CusTrackers.borderList:
                    cv.line(img_to_draw, tuple(border.leftpoint), tuple(border.rightpoint), (0,0,255), 2)
                    cv.circle(img_to_draw, tuple(border.leftpoint), 3, (0, 0, 255),5)
                    cv.circle(img_to_draw, tuple(border.rightpoint), 3, (0, 0, 255),5)
                    cv.putText(img_to_draw, str(border.inoutcount),(border.rightpoint[0], border.rightpoint[1]), cv.FONT_HERSHEY_SIMPLEX,
          1, (0, 0, 255), 3, cv.LINE_AA)
                   # print('CusTrackers.borderConsiderRange',CusTrackers.borderConsiderRange)
                   # print('border.insideArea[2]',border.insideArea[0])
                    #print('border.insideArea[2]',border.insideArea[1])
                    #print('border.insideArea[2]',border.insideArea[2])
                   # print('border.insideArea[2]',border.insideArea[3])
                    cv.circle(img_to_draw, tuple(border.insideArea[0]), 2, (0, 255, 255),5)
                    cv.circle(img_to_draw, tuple(border.insideArea[1]), 2, (0, 255, 255),5)
                    cv.circle(img_to_draw, tuple(border.insideArea[2]), 2, (0, 255, 255),5)
                    cv.circle(img_to_draw, tuple(border.insideArea[3]), 2, (0, 255, 255),5)
                    for insidetrackers in border.insideTrackersHistory:
                        for insidetracker in insidetrackers:
                            
                            print('insidertracker.no',insidetracker.no)
                            print('--')
                        print('------')
                    print('---------------------')
                    for outsidetrackers in border.outsideTrackersHistory:
                        for outsidetracker in outsidetrackers:
                            
                        
                            print('outsidetracker.no',outsidetracker.no)
                            print('--')
                        print('------')
                if cnt > 9:
                    time2 = (time.time() - start ) / 10.
                    start = time.time()
                    cnt = 0
                cv.putText(frame, "tracker = {:.0f} ms".format(time2*1000), (30,60), 2, 0.7, (255,55,155))

                cv.imshow("webCam", img_to_draw)
                out.write(cv.resize(img_to_draw, (1024,1024)))
                #if a == False:
                #    cv.waitKey(0)
                if cv.waitKey(1) & 0xFF == ord('q'):
                    break                
            out.release()

if __name__ == '__main__':

  tf.logging.set_verbosity(tf.logging.INFO)
  tf.app.run()


W0416 17:31:34.456188 140550484342592 deprecation_wrapper.py:119] From /workspace/testbuildmodel/tracking/rotatedSSD.TensorFlow/make_npy.py:21: The name tf.GPUOptions is deprecated. Please use tf.compat.v1.GPUOptions instead.



W0416 17:31:34.456898 140550484342592 deprecation_wrapper.py:119] From /workspace/testbuildmodel/tracking/rotatedSSD.TensorFlow/make_npy.py:22: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



W0416 17:31:34.457509 140550484342592 deprecation_wrapper.py:119] From /workspace/testbuildmodel/tracking/rotatedSSD.TensorFlow/make_npy.py:22: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



W0416 17:31:34.657501 140550484342592 deprecation_wrapper.py:119] From /workspace/testbuildmodel/tracking/rotatedSSD.TensorFlow/make_npy.py:23: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.



priorbox_data = 
(1, 19184, 1)



...

updateTrackers time 136 ms
len at adjusttracker mid 10
CSRT
(322, 315, 46, 46)
nearestcontour == False,i 8
adjustTrackersbyDetection time 13 ms
recordpattern
no 5
no 7
no 8
no 1
no 0
no 4
no 6
no 9
no 10
no 12
retireTrackers
retireoccu
retireTrackers time 11 ms
addtrackers
box [333, 194, 54, 54]
findNumbertoAssign: CusTrackers.numbertoAssignList [1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
CSRT
(333, 194, 54, 54)
box [113, 286, 66, 66]
findNumbertoAssign: CusTrackers.numbertoAssignList [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
CSRT
(113, 286, 66, 66)
addTrackersbyDetection time 18 ms
nms time 0 ms
2 outside no in a row trackerin.no 9
insidertracker.no 12
--
------
insidertracker.no 12
--
------
insidertracker.no 12
--
------
insidertracker.no 12
--
------
insidertracker.no 12
--
------
insidertracker.no 12
--
------
insidertracker.no 12
--
------
------
------
------
---------------------
------
------
------
------
------
------
------
------
------


updateTrackers time 133 ms
len at adjusttracker mid 10
nearestcontour == False,i 4
CSRT
(208, 347, 53, 53)
nearestcontour == False,i 8
adjustTrackersbyDetection time 13 ms
recordpattern
no 5
no 7
no 8
no 1
no 0
no 4
no 6
no 9
no 10
no 12
retireTrackers
retireoccu
retireTrackers time 12 ms
addtrackers
addTrackersbyDetection time 0 ms
nms time 0 ms
2 outside no in a row trackerin.no 9
insidertracker.no 12
--
------
insidertracker.no 12
--
------
------
------
------
------
------
------
------
------
---------------------
------
------
------
------
------
------
------
------
------
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
---------------------
outsidetracker.no 9
--
------
outsidetracker.no 9
--
------
outsidetracker.no 9
--

updateTrackers time 135 ms
len at adjusttracker mid 10
nearestcontour == False,i 1
nearestcontour == False,i 7
nearestcontour == False,i 8
adjustTrackersbyDetection time 4 ms
recordpattern
no 5
no 7
no 8
no 1
no 0
no 4
no 6
no 9
no 10
no 12
retireTrackers
retireoccu
retireTrackers time 12 ms
addtrackers
addTrackersbyDetection time 0 ms
nms time 0 ms
2 outside no in a row trackerin.no 9
------
------
------
------
------
------
------
------
------
------
---------------------
------
------
------
------
------
------
------
------
------
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
insidertracker.no 8
--
------
---------------------
outsidetracker.no 9
--
------
outsidetracker.no 9
--
------
outsidetracker.no 9
--
------
outsidetracker.no 9
--
------
outside

SystemExit: 

/root/miniconda3/envs/tf1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
